<a href="https://colab.research.google.com/github/aborundiya/CNN/blob/master/food_101_InceptionNet86_55.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
downloaded = drive.CreateFile({'id':"1GE6v1Ef9QR4R-SLo_oCx8dUORXvXRNFq"})   # replace the id with id of file you want to access
downloaded.GetContentFile('food-101.zip') 

In [4]:
!unzip food-101.zip

Streaming output truncated to the last 5000 lines.
  inflating: food-101/train/strawberry_shortcake/2245360.jpg  
  inflating: food-101/train/strawberry_shortcake/2259069.jpg  
  inflating: food-101/train/strawberry_shortcake/2259652.jpg  
  inflating: food-101/train/strawberry_shortcake/2266581.jpg  
  inflating: food-101/train/strawberry_shortcake/2271614.jpg  
  inflating: food-101/train/strawberry_shortcake/2275253.jpg  
  inflating: food-101/train/strawberry_shortcake/2282398.jpg  
  inflating: food-101/train/strawberry_shortcake/2282550.jpg  
  inflating: food-101/train/strawberry_shortcake/2288472.jpg  
  inflating: food-101/train/strawberry_shortcake/2291293.jpg  
  inflating: food-101/train/strawberry_shortcake/2292632.jpg  
  inflating: food-101/train/strawberry_shortcake/2300458.jpg  
  inflating: food-101/train/strawberry_shortcake/2302763.jpg  
  inflating: food-101/train/strawberry_shortcake/2307357.jpg  
  inflating: food-101/train/strawberry_shortcake/2312692.jpg  
  in

In [5]:
import numpy as np
import copy 
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.optim as optim 
from torchvision import models
import matplotlib.pyplot as plt 
from google.colab import drive 

#Check for GPU 

In [6]:
device = torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [7]:


train_transforms = transforms.Compose([transforms.RandomResizedCrop(299),
                                       transforms.ColorJitter(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])

valid_transforms =  transforms.Compose([transforms.RandomResizedCrop(299),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(331),
                                      transforms.TenCrop(299),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(crop) for crop in crops]))])


In [8]:
train_set = torchvision.datasets.ImageFolder("./food-101/train",transform=train_transforms)
test_set = torchvision.datasets.ImageFolder("./food-101/test",transform=test_transforms)
valid_set = torchvision.datasets.ImageFolder("./food-101/test",transform = valid_transforms)

In [ ]:
!find food-101/train -type d -or -type f -printf '.' | wc -c
!find food-101/test  -type d -or -type f -printf '.' | wc -c

75750
25250


In [9]:
batch_size = 16 
num_classes = 101
trainloader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=False)
validloader = torch.utils.data.DataLoader(valid_set,batch_size=batch_size,shuffle=False )


# Altering the aux and Fc layer to suit the  dataset num_classes

In [10]:
def initialize_inception(pretrained=True, num_classes=101):
  incepnet = models.inception_v3(pretrained)
  if pretrained : 
    for param in incepnet.parameters():
      param.requires_grad = False
  aux_final_in_features = incepnet.AuxLogits.fc.in_features
  incepnet.AuxLogits.fc = nn.Linear(aux_final_in_features,num_classes)
  final_in_features = incepnet.fc.in_features
  incepnet.fc = nn.Linear(final_in_features,num_classes)
  if pretrained : 
    for param in incepnet.parameters():
      if param.requires_grad:
        print(param.shape)
  return incepnet 

In [11]:
def evalaution_inception(dataloader, model, test=False):
  total , correct = 0 , 0 
  for data in dataloader: 
    inputs, labels = data 
    inputs, labels = inputs.to(device) , labels.to(device)
    if test: 
      #For Multiclass classification TTA validations is summing all class probabilties for the TenCrop and choosing labels using argmax
      #For Regression it will be mean and label prediction it will be mode 
      bs, ncrops, c, h, w = inputs.size()
      outputs = model(inputs.view(-1, c, h, w)) # fuse batch size and ncrops
      outputs = outputs.view(bs, ncrops, -1).sum(1) # Sum over crop
      _ , pred = torch.max(outputs.data,1)
    else :
      outputs = model(inputs)
      _,pred = torch.max(outputs.data,1)

    total += labels.size(0)
    correct += (pred == labels).sum().item()
    del inputs, labels, outputs 
    torch.cuda.empty_cache()
  return 100 * (correct / total )

In [ ]:
inceptnet = initialize_inception()

torch.Size([101, 768])
torch.Size([101])
torch.Size([101, 2048])
torch.Size([101])


#Training and evaluation of the network

In [ ]:
incepnet = inceptnet
incepnet = incepnet.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(incepnet.parameters(), lr = 0.05, momentum=0.9,weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,'min',patience=1,verbose=True,factor=0.9)

In [ ]:

loss_epoch_arr = []
max_epochs = 20
min_loss = 1000
n_iter = np.ceil(75750/batch_size)
valid_loss = 0
for epoch in range(max_epochs):
  incepnet.train()
  loss_arr = 0.0 
  print("Epoch %d / %d " % (epoch, max_epochs))
  for i , data in enumerate(trainloader,0):
    
    inputs, labels = data 
    inputs, labels = inputs.to(device), labels.to(device)
    opt.zero_grad()
    outputs , aux_outputs = incepnet(inputs)
    loss = loss_fn(outputs, labels) + 0.3 * loss_fn(aux_outputs, labels)  # thus to accomodate the aux loss 
    loss.backward()
    opt.step()
    if min_loss > loss : 
      min_loss = loss 
      best_model = copy.deepcopy(incepnet.state_dict())
      #print("Min loss %0.2f" % min_loss) 

    if i % 300 == 0 : 
      print("Epoch %d/%d, Loss : %0.2f, Min Loss : %0.2f"  % (i, n_iter,loss.item(),min_loss))
    loss_arr += loss.item()
    del inputs, labels, outputs, aux_outputs 
    torch.cuda.empty_cache()
  scheduler.step(loss.item())
  loss_epoch_arr.append(loss.item())
  incepnet.eval()
  training_acc = evalaution_inception(trainloader,incepnet)
  valid_acc = evalaution_inception(validloader,incepnet)
  print("Epoch : %d, Avg. Loss : %0.4f  Train Accuracy : %0.2f , Test Accuracy : %0.2f" % (epoch, (loss_arr)/len(trainloader.dataset), training_acc, valid_acc ))
  if valid_acc > valid_loss:
    valid_loss = valid_acc
    best_epoch_model = copy.deepcopy(incepnet.state_dict())

  

Epoch 0 / 20 
Epoch 0/4735, Loss : 6.00, Min Loss : 6.00
Epoch 300/4735, Loss : 7.73, Min Loss : 4.69
Epoch 600/4735, Loss : 8.00, Min Loss : 4.40
Epoch 900/4735, Loss : 6.28, Min Loss : 4.34
Epoch 1200/4735, Loss : 9.40, Min Loss : 4.34
Epoch 1500/4735, Loss : 8.81, Min Loss : 4.34
Epoch 1800/4735, Loss : 11.29, Min Loss : 4.34
Epoch 2100/4735, Loss : 9.28, Min Loss : 4.34
Epoch 2400/4735, Loss : 11.79, Min Loss : 4.34
Epoch 2700/4735, Loss : 9.63, Min Loss : 4.34
Epoch 3000/4735, Loss : 10.81, Min Loss : 4.34
Epoch 3300/4735, Loss : 9.47, Min Loss : 4.34
Epoch 3600/4735, Loss : 12.95, Min Loss : 4.34
Epoch 3900/4735, Loss : 8.00, Min Loss : 4.21
Epoch 4200/4735, Loss : 7.48, Min Loss : 4.21
Epoch 4500/4735, Loss : 9.44, Min Loss : 4.21
Epoch : 0, Avg. Loss : 0.6378  Train Accuracy : 32.43 , Test Accuracy : 34.09
Epoch 1 / 20 
Epoch 0/4735, Loss : 9.85, Min Loss : 2.94
Epoch 300/4735, Loss : 13.24, Min Loss : 2.94
Epoch 600/4735, Loss : 12.20, Min Loss : 2.94
Epoch 900/4735, Loss : 8.

In [ ]:
torch.save(best_model, "bestmodelinceptnet.pt")
torch.save(best_epoch_model,"bestepochmodelinceptnet.pt")

In [ ]:
 torch.cuda.empty_cache()

In [ ]:
print(iter(validloader).next())

In [ ]:

incepnet.eval()
incepnet.load_state_dict(best_model)
print("Epoch %d/%d, Train Accuracy : %0.2f, Test Accuract :%0.2f"% (epoch , max_epochs,
                                                                    evalaution_inception(trainloader,incepnet),
                                                                    evalaution_inception(testloader,incepnet,test=True))) 

Epoch 19/20, Train Accuracy : 38.49, Test Accuract :39.72


In [ ]:
epoch = 19 
max_epochs = 20

In [ ]:

incepnet.eval()
incepnet.load_state_dict(best_epoch_model)
print("Epoch %d/%d, Train Accuracy : %0.2f, Test Accuract :%0.2f"% (epoch , max_epochs,
                                                                    evalaution_inception(trainloader,incepnet),
                                                                    evalaution_inception(testloader,incepnet,test=True))) 

RuntimeError: ignored

In [ ]:
from google.colab import files

In [13]:
incepnet = initialize_inception(pretrained=False)

In [14]:
best_epoch_model = torch.load("bestmodelinceptnet85.pt")

In [15]:
incepnet.load_state_dict(best_epoch_model)
incepnet.to(device)
incepnet.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [16]:


print("Train Accuracy : %0.2f" % (evalaution_inception(trainloader,incepnet)))
print("Test Accuracy without TTA:%0.2f"%(evalaution_inception(validloader,incepnet)))
print("Test Accuracy with TTA :%0.2f"% (evalaution_inception(testloader,incepnet,test=True)))

Train Accuracy : 83.06
Test Accuracy without TTA:79.27
Test Accuracy with TTA :86.55


In [ ]:
incepnet = initialize_inception(pretrained=False)
best_epoch_model = torch.load("inceptionModel20Epochs75.pt")
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(incepnet.parameters(), lr = 0.05, momentum=0.9,weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,'min',patience=1,verbose=True,factor=0.5)
incepnet.load_state_dict(best_epoch_model)

<All keys matched successfully>

In [ ]:
batch_size =16 
loss_epoch_arr = []
max_epochs = 20
min_loss = 1000
valid_loss = 0.0 
n_iter = np.ceil(75750/batch_size)
incepnet.to(device)

for epoch in range(max_epochs):
  incepnet.train()
  loss_arr = 0.0 
  print("Epoch %d / %d " % (epoch, max_epochs))
  for i , data in enumerate(trainloader,0):
    
    inputs, labels = data 
    inputs, labels = inputs.to(device), labels.to(device)
    opt.zero_grad()
    outputs , aux_outputs = incepnet(inputs)
    loss = loss_fn(outputs, labels) + 0.3 * loss_fn(aux_outputs, labels)  # thus to accomodate the aux loss 
    loss.backward()
    opt.step()
    if min_loss > loss : 
      min_loss = loss 
      best_model = copy.deepcopy(incepnet.state_dict())
      #print("Min loss %0.2f" % min_loss) 

    if i % 600 == 0 : 
      print("Epoch %d/%d, Loss : %0.2f, Min Loss : %0.2f"  % (i, n_iter,loss.item(),min_loss))
    loss_arr += loss.item()
    del inputs, labels, outputs, aux_outputs 
    torch.cuda.empty_cache()
  scheduler.step(loss.item())
  loss_epoch_arr.append(loss.item())
  incepnet.eval()
  training_acc = evalaution_inception(trainloader,incepnet)
  valid_acc = evalaution_inception(validloader,incepnet)
  print("Epoch : %d, Avg. Loss : %0.4f  Train Accuracy : %0.2f , Test Accuracy : %0.2f" % (epoch, (loss_arr)/len(trainloader.dataset), training_acc, valid_acc ))
  if valid_acc > valid_loss:
    valid_loss = valid_acc
    best_epoch_model = copy.deepcopy(incepnet.state_dict())


Epoch 0 / 20 
Epoch 0/4735, Loss : 1.98, Min Loss : 1.98
Epoch 600/4735, Loss : 3.96, Min Loss : 0.96
Epoch 1200/4735, Loss : 3.85, Min Loss : 0.96
Epoch 1800/4735, Loss : 3.42, Min Loss : 0.96
Epoch 2400/4735, Loss : 4.31, Min Loss : 0.96
Epoch 3000/4735, Loss : 4.96, Min Loss : 0.96
Epoch 3600/4735, Loss : 3.06, Min Loss : 0.96
Epoch 4200/4735, Loss : 3.68, Min Loss : 0.96
Epoch : 0, Avg. Loss : 0.2286  Train Accuracy : 42.84 , Test Accuracy : 45.10
Epoch 1 / 20 
Epoch 0/4735, Loss : 2.59, Min Loss : 0.96
Epoch 600/4735, Loss : 2.86, Min Loss : 0.96
Epoch 1200/4735, Loss : 3.80, Min Loss : 0.96
Epoch 1800/4735, Loss : 2.92, Min Loss : 0.96
Epoch 2400/4735, Loss : 3.50, Min Loss : 0.96
Epoch 3000/4735, Loss : 3.13, Min Loss : 0.96
Epoch 3600/4735, Loss : 2.17, Min Loss : 0.96
Epoch 4200/4735, Loss : 3.23, Min Loss : 0.96
Epoch : 1, Avg. Loss : 0.1953  Train Accuracy : 43.99 , Test Accuracy : 45.85
Epoch 2 / 20 
Epoch 0/4735, Loss : 3.44, Min Loss : 0.96
Epoch 600/4735, Loss : 2.44, Mi

#Testing

In [ ]:
torch.save(best_model, "bestmodelinceptnet.pt")
torch.save(best_epoch_model,"bestepochmodelinceptnet.pt")

In [ ]:


from google.colab import files
files.download('bestmodelinceptnet.pt')
files.download('bestepochmodelinceptnet.pt')

In [ ]:


print("Epoch %d/%d, Train Accuracy : %0.2f, Test Accuract :%0.2f"% (epoch , max_epochs,
                                                                    evalaution_inception(trainloader,incepnet),
                                                                    evalaution_inception(testloader,incepnet))) 

In [ ]:

incepnet.load_state_dict(best_epoch_model)
incepnet.to(device)
incepnet.eval()

NameError: ignored

In [ ]:

incepnet.eval()
incepnet.load_state_dict(best_epoch_model)
print("Epoch %d/%d, Train Accuracy : %0.2f, Test Accuract :%0.2f"% (epoch , max_epochs,
                                                                    evalaution_inception(trainloader,incepnet),
                                                                    evalaution_inception(testloader,incepnet))) 